In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def generate_random_matrix(n, pos_eigenvalues_count, neg_eigenvalues_count, zero_eigenvalues_count=0, seed=None):
    if seed is not None:
        np.random.seed(seed)
    
    A = np.random.randn(n, n)
    Q, R = np.linalg.qr(A)
    U = Q
    

    D = np.zeros(n)
    
    if pos_eigenvalues_count > 0:
        D[:pos_eigenvalues_count] = np.random.uniform(1, 10, size=pos_eigenvalues_count)
    
    if neg_eigenvalues_count > 0:
        D[pos_eigenvalues_count:pos_eigenvalues_count + neg_eigenvalues_count] = -np.random.uniform(1, 10, size=neg_eigenvalues_count)
    
    C = U @ np.diag(D) @ U.T
    
    return C, D

def compute_frobenius_error(C, M):
    error = np.linalg.norm(C - M, 'fro')**2
    return error

def best_rank_r_approximation(C, r):
    eigenvalues, eigenvectors = np.linalg.eigh(C)
    
    sorted_indices = np.argsort(np.abs(eigenvalues))[::-1]
    top_indices = sorted_indices[:r]
    top_eigenvalues = eigenvalues[top_indices]
    top_eigenvectors = eigenvectors[:, top_indices]
    
    M_r = top_eigenvectors @ np.diag(top_eigenvalues) @ top_eigenvectors.T
    return M_r

def plot_errors(errors, pos_eigenvalues_count):
    n = len(errors)
    r_values = range(1, n+1)
    plt.figure(figsize=(10, 6))
    plt.plot(r_values, errors, marker='o', linestyle='-', label='Frobenius Norm Error')
    plt.axvline(x=pos_eigenvalues_count, color='r', linestyle='--', label=f'Positive Eigenvalues = {pos_eigenvalues_count}')
    plt.title('Frobenius Norm Error vs Rank r')
    plt.xlabel('Rank r')
    plt.ylabel('||C - M_r||_F^2')
    plt.legend()
    plt.grid(True)
    plt.show()

def plot_error_rate_of_change(error_rates, pos_eigenvalues_count):
    n = len(error_rates)
    r_values = range(2, n+2)
    plt.figure(figsize=(10, 6))
    plt.plot(r_values, error_rates, marker='o', linestyle='-', color='g', label='Error Rate of Change')
    plt.axvline(x=pos_eigenvalues_count, color='r', linestyle='--', label=f'Positive Eigenvalues = {pos_eigenvalues_count}')
    plt.title('Error Rate of Change vs Rank r')
    plt.xlabel('Rank r')
    plt.ylabel('Δ||C - M_r||_F^2')
    plt.legend()
    plt.grid(True)
    plt.show()

def simulation_study(n, pos_eigenvalues_count, neg_eigenvalues_count, zero_eigenvalues_count=0, num_simulations=100, seed=None):
    if seed is not None:
        np.random.seed(seed)
    
    cumulative_errors = np.zeros(n)
    
    for sim in range(num_simulations):
        C, D = generate_random_matrix(n, pos_eigenvalues_count, neg_eigenvalues_count, zero_eigenvalues_count)
        errors = []
        for r in range(1, n+1):
            M_r = best_rank_r_approximation(C, r)
            error = compute_frobenius_error(C, M_r)
            errors.append(error)
        cumulative_errors += np.array(errors)
    
    avg_errors = cumulative_errors / num_simulations
    return avg_errors

def calculate_error_rate_of_change(avg_errors):
    error_rates = []
    for i in range(1, len(avg_errors)):
        delta_error = avg_errors[i-1] - avg_errors[i]
        error_rates.append(delta_error)
    return error_rates


n = 100
pos_eigenvalues_count = 70
neg_eigenvalues_count = 27
zero_eigenvalues_count = n - pos_eigenvalues_count - neg_eigenvalues_count
seed = 42
num_simulations = 100

avg_errors = simulation_study(n, pos_eigenvalues_count, neg_eigenvalues_count, zero_eigenvalues_count, num_simulations, seed)

error_rates = calculate_error_rate_of_change(avg_errors)

print("Average Frobenius Norm Errors:")
for r, error in zip(range(1, n+1), avg_errors):
    print(f"Rank {r}: ||C - M_r||_F^2 = {error:.4f}")

print("\nError Rate of Change:")
for r, delta_error in zip(range(1, n), error_rates):
    print(f"Δ||C - M_r||_F^2 (r={r} to r={r+1}) = {delta_error:.4f}")

plot_errors(avg_errors, pos_eigenvalues_count)

plot_error_rate_of_change(error_rates, pos_eigenvalues_count)


KeyboardInterrupt: 